# NEARCoder2-3B compared evaluation

The NEARCoder is built on StarCoder2-3B. In this notebook, we are evaluating the model performance by comparing it with the baseline model and each of the building steps:
- StarCoder2-3B
- NEAR-preTrainedStarCoder2
- NEAR-structTunedStarCoder2
- NEARCoder2-3b

Our primary focus will be to ensure that the model doesn't lose performance to general coding tasks, and that it increases performance on the specialized coding tasks of the NEAR protocol.